<img src="/Users/adrianselk/Desktop/DHBW-Projekte/data_exploration_project/img/tom-jerry-min_cf9c8a43-acdd-4e46-b8b1-72972a850258.webp" 
     alt="header" width="100%" height="180">


# Data Exploration Project: Tom & Jerry Image Classification

In [10]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Aktuelles Gerät:", device)

Aktuelles Gerät: mps
